In [1]:
x=2

In [2]:
%load_ext autoreload
%autoreload 2

In [17]:
import scanpy as sc
import pandas as pd
import multigrate as mtg
import multimil as mtm
from pathlib import Path
import shutil

from matplotlib import pyplot as plt
import scanpy as sc
import pandas as pd
import anndata as ad
import os
import random
import torch
import scvi
import numpy as np

In [4]:
adata = sc.read('../run_baselines/data/multigrate_mil/41301bf6a3/0/checkpoints/train_anndata.h5ad')
adata

AnnData object with n_obs × n_vars = 438582 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_leve

In [5]:
idx = adata.obs['sample'].sort_values().index
adata = adata[idx].copy()

In [6]:
adata = mtg.data.organize_multiome_anndatas(
            adatas = [[adata]],
        )

/tmp/ipykernel_2758079/3981741654.py:1: UserWarning: Column `.obs['group']` will be overwritten. Please save the original data in another column if needed.
  adata = mtg.data.organize_multiome_anndatas(


In [7]:
adata

AnnData object with n_obs × n_vars = 438582 × 30
    obs: "3'_or_5'", 'BMI', 'age_or_mean_of_age_range', 'age_range', 'anatomical_region_ccf_score', 'ancestry', 'ann_coarse_for_GWAS_and_modeling', 'ann_finest_level', 'ann_level_1', 'ann_level_2', 'ann_level_3', 'ann_level_4', 'ann_level_5', 'assay', 'cause_of_death', 'cell_type', 'core_or_extension', 'dataset', 'development_stage', 'disease', 'donor_id', 'fresh_or_frozen', 'log10_total_counts', 'lung_condition', 'mixed_ancestry', 'original_ann_level_1', 'original_ann_level_2', 'original_ann_level_3', 'original_ann_level_4', 'original_ann_level_5', 'original_ann_nonharmonized', 'reannotation_type', 'sample', 'scanvi_label', 'sequencing_platform', 'sex', 'smoking_status', 'study', 'subject_type', 'suspension_type', 'tissue', 'tissue_coarse_unharmonized', 'tissue_detailed_unharmonized', 'tissue_dissociation_protocol', 'tissue_level_2', 'tissue_level_3', 'tissue_sampling_method', 'total_counts', 'transf_ann_level_1_label', 'transf_ann_leve

In [8]:
mtm.model.MILClassifier.setup_anndata(
    adata, 
    rna_indices_end=30,
    categorical_covariate_keys=['sample', 'lung_condition_coarse'],
)

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [9]:
mil = mtm.model.MILClassifier(
            adata, 
            classification=[
                'lung_condition_coarse'
            ],
            patient_label='sample',
            class_loss_coef=1.0,
            z_dim=30,
        )

In [10]:
mil.train(
    lr=1e-4,
    batch_size=512, 
    max_epochs=2,
)

/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.10 /lustre/groups/ml01/workspace/anastasia.litinets ...
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/lightning_fabric/plugins/environments/slurm.py:165: PossibleUserWarning: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python3.10 /lustre/groups/ml01/workspace/anastasia.litinets ..

Epoch 2/2: 100%|██████████| 2/2 [00:30<00:00, 14.11s/it, loss=0.225, v_num=1]

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 2/2: 100%|██████████| 2/2 [00:30<00:00, 15.45s/it, loss=0.225, v_num=1]


In [14]:
mil.get_model_output(batch_size=512)

INFO     Input AnnData not setup with scvi-tools. attempting to transfer AnnData setup                             


/lustre/groups/ml01/workspace/anastasia.litinetskaya/mambaforge/envs/multimil/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:90: UserWarning: adata.X does not contain unnormalized count data. Are you sure this is what you want?
  warnings.warn(


In [15]:
adata.obs['bags']

222C_ATTATCCAGTAATCCC_adams       0
222C_TCAACGATCCGCAAGC_adams       0
222C_ATTACTCTCCAAATGC_adams       0
222C_GGCAATTAGTACACCT_adams       0
222C_CATCAGAGTACCTACA_adams       0
                               ... 
muc9832_TAACTTTTAGTC_mayr      3293
muc9832_GCCGCCAGGTTG_mayr      3293
muc9832_CTCCAAGGCGGC_mayr      3293
muc9832_CTCTTTTGAAGC_mayr      3293
muc9832_TGTTCATACCTC_mayr      3293
Name: bags, Length: 438582, dtype: int64